###  Dataset
---
We used this [face dataset](https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/data) which consists of 48x48 grayscale images of faces with labels in one of 7 categories: 0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral.

An example of a face in this dataset:
<img src = Images/ExampleFace.jpg alt = "Example Face" width =200>


In [21]:
#imports
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA, NMF, KernelPCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
# fix random seed for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)

In [3]:
# reading and cleaning data
total = pd.read_csv('../data/raw/icml_face_data.csv')
total = total.drop(columns = [' Usage'])

total_y = total['emotion']
total_x = total[' pixels'].str.split(' ',expand=True).astype(dtype = 'uint8')

x_train, x_test, y_train, y_test  = train_test_split(total_x, total_y, test_size=0.25, random_state=42)

print("Training Components: "+ str(y_train.size))
print("Test Components: "+ str(y_test.size))

Training Components: 26915
Test Components: 8972


In [4]:
# fitting data
sc = MinMaxScaler()

x_train_sc = sc.fit_transform(x_train)
x_test_sc = sc.transform(x_test)


###  PCA

In [5]:
# applying pca

pca = PCA(n_components=255).fit(x_train_sc) #n_components = (255 explains 95%) (881 explains 99%)
x_train_pca = pca.transform(x_train_sc)
x_test_pca = pca.transform(x_test_sc)

explained_variance = pca.explained_variance_ratio_

### NMF

In [15]:
# apply nmf
nmf = NMF(n_components=255, init='nndsvd', random_state=seed)
x_train_nmf = nmf.fit_transform(x_train_sc)
x_test_nmf = nmf.transform(x_test_sc)
H = nmf.components_

/data/code/bsrikish/linalg/my_venv/lib/python3.7/site-packages/sklearn/decomposition/_nmf.py:1091: ConvergenceWarning: Maximum number of iterations 10 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)
/data/code/bsrikish/linalg/my_venv/lib/python3.7/site-packages/sklearn/decomposition/_nmf.py:1091: ConvergenceWarning: Maximum number of iterations 10 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)


### Kernel PCA

In [25]:
kpca = KernelPCA(n_components=7, kernel='rbf', random_state=seed)
x_train_kpca = kpca.fit_transform(x_train_sc)
x_test_kpca = kpca.transform(x_test_sc)

### Logistic Regression Results

In [9]:
# logistic regression on pca representation
lrc = LogisticRegression(C=1.0, penalty = 'l2', solver = 'saga', tol=0.0001, max_iter=800, random_state=seed)
lrc.fit(x_train_pca, y_train)
predictions = lrc.predict(x_test_pca)

print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions, zero_division=0))

[[ 193    4  105  411  247   80  195]
 [  16    6   27   37   18    9   17]
 [ 116    3  187  401  221  175  201]
 [ 106    1   93 1543  240  102  163]
 [ 149    2  109  448  428   80  283]
 [  38    2  112  212   74  446  106]
 [  89    2   86  482  271  109  527]]
              precision    recall  f1-score   support

           0       0.27      0.16      0.20      1235
           1       0.30      0.05      0.08       130
           2       0.26      0.14      0.18      1304
           3       0.44      0.69      0.53      2248
           4       0.29      0.29      0.29      1499
           5       0.45      0.45      0.45       990
           6       0.35      0.34      0.34      1566

    accuracy                           0.37      8972
   macro avg       0.34      0.30      0.30      8972
weighted avg       0.35      0.37      0.35      8972



In [ ]:
# logistic regression on nmf representation
lrc = LogisticRegression(C=0.01, penalty = 'l2', solver = 'saga', tol=0.0001, max_iter=800, random_state=seed)
lrc.fit(x_train_nmf, y_train)
predictions = lrc.predict(x_test_nmf)

print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions, zero_division=0))

In [26]:
# logistic regression on kernel pca representation
lrc = LogisticRegression(C=0.01, penalty = 'l2', solver = 'saga', tol=0.0001, max_iter=800, random_state=seed)
lrc.fit(x_train_kpca, y_train)
predictions = lrc.predict(x_test_kpca)

print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions, zero_division=0))

[[   0    0    0 1229    6    0    0]
 [   0    0    0  129    1    0    0]
 [   0    0    0 1301    3    0    0]
 [   0    0    0 2245    3    0    0]
 [   0    0    0 1493    6    0    0]
 [   0    0    0  989    1    0    0]
 [   0    0    0 1563    3    0    0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1235
           1       0.00      0.00      0.00       130
           2       0.00      0.00      0.00      1304
           3       0.25      1.00      0.40      2248
           4       0.26      0.00      0.01      1499
           5       0.00      0.00      0.00       990
           6       0.00      0.00      0.00      1566

    accuracy                           0.25      8972
   macro avg       0.07      0.14      0.06      8972
weighted avg       0.11      0.25      0.10      8972



### Support Vector Machine Classifier Results

In [24]:
svc = LinearSVC(random_state=seed)
svc.fit(x_train_pca, y_train)
predictions = svc.predict(x_test_pca)

print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions, zero_division=0))

[[ 132    0   56  533  184  134  196]
 [  12    0   19   54   11   15   19]
 [  79    0  109  492  185  226  213]
 [  64    0   46 1696  183  116  143]
 [  95    0   56  577  342  122  307]
 [  18    0   54  247   72  501   98]
 [  61    0   41  593  210  155  506]]
              precision    recall  f1-score   support

           0       0.29      0.11      0.16      1235
           1       0.00      0.00      0.00       130
           2       0.29      0.08      0.13      1304
           3       0.40      0.75      0.53      2248
           4       0.29      0.23      0.25      1499
           5       0.39      0.51      0.44       990
           6       0.34      0.32      0.33      1566

    accuracy                           0.37      8972
   macro avg       0.29      0.29      0.26      8972
weighted avg       0.33      0.37      0.32      8972



/data/code/bsrikish/linalg/my_venv/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
svc = LinearSVC(random_state=seed)
svc.fit(x_train_nmf, y_train)
predictions = svc.predict(x_test_nmf)

print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions, zero_division=0))

In [ ]:
svc = LinearSVC(random_state=seed)
svc.fit(x_train_kpca, y_train)
predictions = svc.predict(x_test_kpca)

print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions, zero_division=0))

### MLP Results

In [16]:
mlp = MLPClassifier(random_state=seed, max_iter=300)
mlp.fit(x_train_pca, y_train)
predictions = mlp.predict(x_test_pca)

print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions, zero_division=0))

[[ 371   24  184  178  224   80  174]
 [  13   43   15   13   20    6   20]
 [ 178   17  376  186  235  133  179]
 [ 213   22  204 1164  254  113  278]
 [ 221   16  242  256  447   81  236]
 [  65   16  127   92   69  534   87]
 [ 196   20  171  293  289  109  488]]
              precision    recall  f1-score   support

           0       0.30      0.30      0.30      1235
           1       0.27      0.33      0.30       130
           2       0.29      0.29      0.29      1304
           3       0.53      0.52      0.53      2248
           4       0.29      0.30      0.29      1499
           5       0.51      0.54      0.52       990
           6       0.33      0.31      0.32      1566

    accuracy                           0.38      8972
   macro avg       0.36      0.37      0.36      8972
weighted avg       0.38      0.38      0.38      8972



/data/code/bsrikish/linalg/my_venv/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [ ]:
mlp = MLPClassifier(random_state=seed, max_iter=300)
mlp.fit(x_train_nmf, y_train)
predictions = mlp.predict(x_test_nmf)

print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions, zero_division=0))

In [ ]:
mlp = MLPClassifier(random_state=seed, max_iter=300)
mlp.fit(x_train_kpca, y_train)
predictions = mlp.predict(x_test_kpca)

print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions, zero_division=0))

### Random Forest Classifier

In [20]:
rfc = RandomForestClassifier(random_state=seed)
rfc.fit(x_train_pca, y_train)
predictions = rfc.predict(x_test_pca)

print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions, zero_division=0))

[[ 200    0   50  663  170   21  131]
 [   6   33    7   54   14    2   14]
 [  50    0  290  642  167   41  114]
 [  33    0   56 1855  156   20  128]
 [  57    0   70  820  360    8  184]
 [  26    0   52  345   61  424   82]
 [  44    0   61  862  187   15  397]]
              precision    recall  f1-score   support

           0       0.48      0.16      0.24      1235
           1       1.00      0.25      0.40       130
           2       0.49      0.22      0.31      1304
           3       0.35      0.83      0.50      2248
           4       0.32      0.24      0.28      1499
           5       0.80      0.43      0.56       990
           6       0.38      0.25      0.30      1566

    accuracy                           0.40      8972
   macro avg       0.55      0.34      0.37      8972
weighted avg       0.45      0.40      0.37      8972



In [ ]:
rfc = RandomForestClassifier(random_state=seed)
rfc.fit(x_train_nmf, y_train)
predictions = rfc.predict(x_test_nmf)

print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions, zero_division=0))

In [ ]:
rfc = RandomForestClassifier(random_state=seed)
rfc.fit(x_train_kpca, y_train)
predictions = rfc.predict(x_test_kpca)

print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions, zero_division=0))

### K nearest neighbors Classifier

In [23]:
knc = KNeighborsClassifier()
knc.fit(x_train_pca, y_train)
predictions = knc.predict(x_test_pca)

print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions, zero_division=0))

[[ 370   19  134  288  167   65  192]
 [  17   48    9   30   10    5   11]
 [ 167   26  378  281  169  100  183]
 [ 233   42  222 1065  201  114  371]
 [ 207   13  178  401  378   52  270]
 [ 105   19  118  183   79  368  118]
 [ 189   28  138  422  198   90  501]]
              precision    recall  f1-score   support

           0       0.29      0.30      0.29      1235
           1       0.25      0.37      0.30       130
           2       0.32      0.29      0.30      1304
           3       0.40      0.47      0.43      2248
           4       0.31      0.25      0.28      1499
           5       0.46      0.37      0.41       990
           6       0.30      0.32      0.31      1566

    accuracy                           0.35      8972
   macro avg       0.33      0.34      0.33      8972
weighted avg       0.35      0.35      0.34      8972



In [ ]:
knc = KNeighborsClassifier()
knc.fit(x_train_nmf, y_train)
predictions = knc.predict(x_test_nmf)

print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions, zero_division=0))

In [ ]:
knc = KNeighborsClassifier()
knc.fit(x_train_kpca, y_train)
predictions = knc.predict(x_test_kpca)

print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions, zero_division=0))